In [5]:
# !pip install deepspeed

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.2 MB/s eta 0:00:00
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
  Created wheel for deepspeed: filename=deepspeed-0.14.4-py3-none-any.whl size=1445513 sha256=919fb150ce700c2ad243896858939f703fc351f2bde7f7f4329fe2fa29811e9c
  Stored in directory: /home/jupyter/.cache/pip/wheels/8e/bc/a3/608e90bbb301848b78fd75d24d6d43ba3074de968fc0e397ac
Successfully built deepspeed


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import deepspeed

# model_id = "EleutherAI/gpt-neo-1.3B"

# # load model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).cuda()

# payload = "I want a movie that is a mix between The Internship with Owen Wilson and Vince Vaughan and Iron Man with Robert Downey Jnr."

# input_ids = tokenizer(payload,return_tensors="pt").input_ids.to(model.device)
# print(f"input payload: \n \n{payload}")
# logits = model.generate(input_ids, do_sample=True, num_beams=1, min_length=200, max_new_tokens=200)

# print(f"prediction: \n \n {tokenizer.decode(logits[0].tolist()[len(input_ids[0]):])}")

from time import perf_counter
import numpy as np
import transformers
# hide generation warnings
transformers.logging.set_verbosity_error()


[2024-07-23 21:05:14,947] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [2]:
import pandas as pd
import gc
from torch.utils.data import Dataset, random_split

torch.manual_seed(42)
texts = pd.read_csv('data.csv')
texts = texts.dropna()
texts = texts.dropna(subset=['Quest Description'])
# print(texts)
# print(texts['Quest Description'])
class TextDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.labels = []
        self.input_ids = []
        self.attn_masks = []        
        for title, descrp in zip(txt_list['Quest Title'], txt_list['Quest Description']):
            encodings_dict = tokenizer('<|startoftext|>' + 'Title: ' + title + 'Description: ' + descrp + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    def __len__(self): return len(self.input_ids)
    def __getitem__(self, idx): return self.input_ids[idx], self.attn_masks[idx]

In [3]:
# Set the random seed to a fixed value to get reproducible results 
torch.manual_seed(42)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B",    
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')


if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


max_length = max([len(tokenizer.encode('Title: ' + title + 'Description: ' + descr)) for title, descr in zip(texts['Quest Description'], texts['Quest Title'])])
max_length2 = max([len(tokenizer.encode('<|startoftext|>' + 'Title: ' + title + 'Description: ' + descr + '<|endoftext|>')) for title, descr in zip(texts['Quest Description'], texts['Quest Title'])])
print(max_length)
print(max_length2)
dataset = TextDataset(texts, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
gc.collect()
print(train_size)

model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
model.resize_token_embeddings(len(tokenizer))

# Initialize the accelerator
from accelerate import Accelerator
accelerator = Accelerator()
print(f"Using {accelerator.device} device")

training_args = TrainingArguments(output_dir='./results', num_train_epochs=1, logging_steps=10, save_strategy="epoch", eval_strategy='epoch',
                                  per_device_train_batch_size=8, per_device_eval_batch_size=8, warmup_steps=100, gradient_accumulation_steps=2,
                                  weight_decay=0.01, logging_dir='./logs', fp16=True, deepspeed='./ds_config_gpt_j.json')
from accelerate.utils import DistributedType
training_args.distributed_state.distributed_type = DistributedType.DEEPSPEED

# print(f"Gradient Accumulation Steps: {training_args.gradient_accumulation_steps}")
# print(f"Distributed Type: {training_args.distributed_state.distributed_type}")
# print(training_args)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})
# Add print statements in the DeepSpeed preparation function (Monkey patching)
from accelerate.accelerator import Accelerator

def patched_prepare_deepspeed(self, *args):
    batch_size_per_device = self._train_batch_size
    gradient_accumulation_steps = self.gradient_accumulation_steps
    num_processes = self.num_processes

    # Debugging: Print the values being used
    print(f"batch_size_per_device: {batch_size_per_device}")
    print(f"gradient_accumulation_steps: {gradient_accumulation_steps}")
    print(f"num_processes: {num_processes}")

    config_kwargs = {
        "train_micro_batch_size_per_gpu": batch_size_per_device,
        "train_batch_size": batch_size_per_device * gradient_accumulation_steps * num_processes,
        "gradient_clipping": 1.0,
        "zero_optimization.stage3_gather_16bit_weights_on_model_save": False,
    }

    # Debugging: Print the final configuration
    print(f"DeepSpeed Configuration: {config_kwargs}")

    model = None
    optimizer = None

    return model, optimizer, args

Accelerator._prepare_deepspeed = patched_prepare_deepspeed

trainer.accelerator = accelerator

trainer.train()
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                bos_token='<|startoftext|>',
                                eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

173
175
6096
Using cuda device


AttributeError: 'Accelerator' object has no attribute '_train_batch_size'

In [4]:
# !pip show transformers accelerate deepspeed
import logging
logging.basicConfig(level=logging.DEBUG)
